In [27]:
# Make a copick project
import os
import shutil

config_blob = """{
    "name": "czii_cryoet_mlchallenge_2024",
    "description": "2024 CZII CryoET ML Challenge training data.",
    "version": "1.0.0",

    "pickable_objects": [
        {
            "name": "apo-ferritin",
            "is_particle": true,
            "pdb_id": "4V1W",
            "label": 1,
            "color": [  0, 117, 220, 128],
            "radius": 60,
            "map_threshold": 0.0418
        },
        {
          "name" : "beta-amylase",
            "is_particle": true,
            "pdb_id": "8ZRZ",
            "label": 2,
            "color": [255, 255, 255, 128],
            "radius": 90,
            "map_threshold": 0.0578  
        },
        {
            "name": "beta-galactosidase",
            "is_particle": true,
            "pdb_id": "6X1Q",
            "label": 3,
            "color": [ 76,   0,  92, 128],
            "radius": 90,
            "map_threshold": 0.0578
        },
        {
            "name": "ribosome",
            "is_particle": true,
            "pdb_id": "6EK0",
            "label": 4,
            "color": [  0,  92,  49, 128],
            "radius": 150,
            "map_threshold": 0.0374
        },
        {
            "name": "thyroglobulin",
            "is_particle": true,
            "pdb_id": "6SCJ",
            "label": 5,
            "color": [ 43, 206,  72, 128],
            "radius": 130,
            "map_threshold": 0.0278
        },
        {
            "name": "virus-like-particle",
            "is_particle": true,
            "label": 6,
            "color": [255, 204, 153, 128],
            "radius": 135,
            "map_threshold": 0.201
        },
        {
            "name": "membrane",
            "is_particle": false,
            "label": 8,
            "color": [100, 100, 100, 128]
        },
        {
            "name": "background",
            "is_particle": false,
            "label": 9,
            "color": [10, 150, 200, 128]
        }
    ],

    "overlay_root": "./kaggle/working/overlay",

    "overlay_fs_args": {
        "auto_mkdir": true
    },

    "static_root": "./kaggle/input/czii-cryo-et-object-identification/train/static"
}"""

copick_config_path = "./kaggle/working/copick.config"
output_overlay = "./kaggle/working/overlay"


with open(copick_config_path, "w") as f:
    f.write(config_blob)
    
# Update the overlay
# Define source and destination directories
source_dir = './kaggle/input/czii-cryo-et-object-identification/train/overlay'
destination_dir = './kaggle/working/overlay'

# Walk through the source directory
for root, dirs, files in os.walk(source_dir):
    # Create corresponding subdirectories in the destination
    relative_path = os.path.relpath(root, source_dir)
    target_dir = os.path.join(destination_dir, relative_path)
    os.makedirs(target_dir, exist_ok=True)
    
    # Copy and rename each file
    for file in files:
        if file.startswith("curation_0_"):
            new_filename = file
        else:
            new_filename = f"curation_0_{file}"
            
        
        # Define full paths for the source and destination files
        source_file = os.path.join(root, file)
        destination_file = os.path.join(target_dir, new_filename)
        
        # Copy the file with the new name
        shutil.copy2(source_file, destination_file)
        print(f"Copied {source_file} to {destination_file}")

Copied ./kaggle/input/czii-cryo-et-object-identification/train/overlay/ExperimentRuns/TS_5_4/Picks/ribosome.json to ./kaggle/working/overlay/ExperimentRuns/TS_5_4/Picks/curation_0_ribosome.json
Copied ./kaggle/input/czii-cryo-et-object-identification/train/overlay/ExperimentRuns/TS_5_4/Picks/virus-like-particle.json to ./kaggle/working/overlay/ExperimentRuns/TS_5_4/Picks/curation_0_virus-like-particle.json
Copied ./kaggle/input/czii-cryo-et-object-identification/train/overlay/ExperimentRuns/TS_5_4/Picks/beta-galactosidase.json to ./kaggle/working/overlay/ExperimentRuns/TS_5_4/Picks/curation_0_beta-galactosidase.json
Copied ./kaggle/input/czii-cryo-et-object-identification/train/overlay/ExperimentRuns/TS_5_4/Picks/beta-amylase.json to ./kaggle/working/overlay/ExperimentRuns/TS_5_4/Picks/curation_0_beta-amylase.json
Copied ./kaggle/input/czii-cryo-et-object-identification/train/overlay/ExperimentRuns/TS_5_4/Picks/apo-ferritin.json to ./kaggle/working/overlay/ExperimentRuns/TS_5_4/Picks/c

In [43]:
import copick
import numpy as np
from tqdm import tqdm


root = copick.from_file(copick_config_path)

copick_user_name = "copickUtils"
copick_segmentation_name = "paintedPicks"
voxel_size = 10
tomo_tpye_list = ["ctfdeconvolved","denoised","isonetcorrected","wbp"]
tomo_type = tomo_tpye_list[1]
print(copick_config_path)



./kaggle/working/copick.config


/var/folders/l1/vt4hj2ln3hs6c97_qllsy7lh0000gn/T/ipykernel_38456/4283015292.py:6: DeprecationWarning: config_type not found in config file, defaulting to filesystem
  root = copick.from_file(copick_config_path)


In [44]:
from copick_utils.src.copick_utils.segmentation import segmentation_from_picks
import copick_utils.src.copick_utils.writers.write as write
from collections import defaultdict

# Just do this once
generate_masks = True

if generate_masks:
    target_objects = defaultdict(dict)
    for object in root.pickable_objects:
        if object.is_particle:
            target_objects[object.name]['label'] = object.label
            target_objects[object.name]['radius'] = object.radius


    for run in tqdm(root.runs):
        tomo = run.get_voxel_spacing(10)
        tomo = tomo.get_tomogram(tomo_type).numpy()
        target = np.zeros(tomo.shape, dtype=np.uint8)
        for pickable_object in root.pickable_objects:
            pick = run.get_picks(object_name=pickable_object.name, user_id="curation")
            if len(pick):  
                target = segmentation_from_picks.from_picks(pick[0], 
                                                            target, 
                                                            target_objects[pickable_object.name]['radius'] * 0.8, # 3d Mask Size = Radius in mask
                                                            target_objects[pickable_object.name]['label'] # label_value
                                                            )
        write.segmentation(run, target, copick_user_name, name=copick_segmentation_name)

data_dicts = []
for run in tqdm(root.runs):
    tomogram = run.get_voxel_spacing(voxel_size).get_tomogram(tomo_type).numpy()
    segmentation = run.get_segmentations(name=copick_segmentation_name, user_id=copick_user_name, voxel_size=voxel_size, is_multilabel=True)[0].numpy()
    data_dicts.append({"image": tomogram, "label": segmentation})
    


  0%|          | 0/7 [00:00<?, ?it/s]

100%|██████████| 7/7 [00:01<00:00,  4.24it/s]


In [30]:
for i in range(7):
    print(data_dicts[i]["image"].shape, data_dicts[i]["label"].shape, np.unique(data_dicts[i]["label"]))

(184, 630, 630) (184, 630, 630) [0 1 2 3 4 5 6]
(184, 630, 630) (184, 630, 630) [0 1 2 3 4 5 6]
(184, 630, 630) (184, 630, 630) [0 1 2 3 4 5 6]
(184, 630, 630) (184, 630, 630) [0 1 2 3 4 5 6]
(184, 630, 630) (184, 630, 630) [0 1 2 3 4 5 6]
(184, 630, 630) (184, 630, 630) [0 1 2 3 4 5 6]
(184, 630, 630) (184, 630, 630) [0 1 2 3 4 5 6]


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

for i in range(7):
    # 형태와 unique 값 출력
    print(f"Shape - Image: {data_dicts[i]['image'].shape}, Label: {data_dicts[i]['label'].shape}")
    unique_labels = np.unique(data_dicts[i]["label"])
    print(f"Unique labels: {unique_labels}")
    
    # 시각화
    plt.figure(figsize=(15, 5))
    
    # 원본 이미지와 마스크 나란히 표시
    plt.subplot(1, 3, 1)
    plt.title('Tomogram')
    plt.imshow(data_dicts[i]['image'][100], cmap='gray')
    plt.axis('off')
    
    plt.subplot(1, 3, 2)
    plt.title('Segmentation Mask')
    mask = plt.imshow(data_dicts[i]['label'][100], cmap='viridis')
    plt.colorbar(mask, label='Label Values')
    plt.axis('off')
    
    # 겹친 이미지 표시
    plt.subplot(1, 3, 3)
    plt.title('Overlay')
    plt.imshow(data_dicts[i]['image'][100], cmap='gray')
    plt.imshow(data_dicts[i]['label'][100], cmap='viridis', alpha=0.5)
    plt.colorbar(label='Label Values')
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()